In [1]:
import os

list_file = sorted(list(os.walk('./4.model/'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['make_102.Land Rover-type_1_error.xlsx']

In [2]:
file = list_file[0]

file

'make_102.Land Rover-type_1_error.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./4.model/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['Make No'] = input_['Make No'].astype(int)
input_['Model No'] = input_['Model No'].astype(int)
input_['Year'] = input_['Year'].astype(int)

print(f'总数量：{len(input_)}')
input_['Model'].tolist()

总数量：1


['Range Rover Sport']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [ ]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'cssClass': 'form-control',
        'dropdownType': '3',
        'selectedValue': '',
        'vehicleAttributes': {'Make': f'''{input_.loc[a, 'Make']}''',
                              'MakeID': f'''{input_.loc[a, 'Make Code']}''',
                              'ModelID': f'''{input_.loc[a, 'Model Code']}''',
                              'SkinID': '1',
                              'Year': f'''{input_.loc[a, 'Year']}'''}}

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                             data=json.dumps(data),
                             headers=get_header(**{'Content-Type': 'application/json; charset=UTF-8'}),
                             proxies=get_proxy(),
                             timeout=(10, 10))
        
        if resp.status_code == 200:
            break
    except:
        continue

resp

In [ ]:
json_ = resp.json()

json_

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['d'], 'lxml')
html = etree.HTML(str(soup))

with open('./type_1.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

In [ ]:
list_type_code = html.xpath('//select[@id="parttypeDropdown"]/option/@value')[1:]

list_type_code

In [ ]:
list_type = html.xpath('//select[@id="parttypeDropdown"]/option/text()')[1:]

list_type

In [ ]:
len(list_type) == 0

In [ ]:
df_temp = pd.DataFrame({'Make No': input_.loc[a, 'Make No'],
                        'Make': input_.loc[a, 'Make'],
                        'Make Code': input_.loc[a, 'Make Code'],
                        'Model No': input_.loc[a, 'Model No'],
                        'Model': input_.loc[a, 'Model'],
                        'Model Code': input_.loc[a, 'Model Code'],
                        'Year': input_.loc[a, 'Year'],
                        'Type No': [i+1 for i in range(len(list_type))],
                        'Type': list_type,
                        'Type Code': list_type_code,
                        'Type Url': [f'https://www.dormanproducts.com/gsearch.aspx?type=keyword&q={type_}' for type_ in list_type]})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

In [ ]:
crawler_status = 'ok'

crawler_status

In [ ]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Make No': input_.loc[a, 'Make No'],
                             'Make': input_.loc[a, 'Make'],
                             'Make Code': input_.loc[a, 'Make Code'],
                             'Model No': input_.loc[a, 'Model No'],
                             'Model': input_.loc[a, 'Model'],
                             'Model Code': input_.loc[a, 'Model Code'],
                             'Year': input_.loc[a, 'Year']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

In [ ]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make No', 'Model No', 'Year', 'Type No'],
                                                ascending=[True, True, False, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_type_1.xlsx', index=False)

output_correct

In [ ]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make No', 'Model No', 'Year'],
                                            ascending=[True, True, False],
                                            ignore_index=True)
    output_error.to_excel('./test_type_1_error.xlsx', index=False)

output_error